**Configuration File Details:**
The configuration file (`nz.yml`) specifies the following for the dataset:
- **Data Type:** custom, suitable for structured input/output format.
- **Tokenizer:** LlamaTokenizer with specific special tokens.

**Dataset Structure:**
- **Fields:** "instruction," "input," "output".
- **Example:**
  ```json
  {
      "instruction": "Claim coders manages claims...",
      "input": "IncidentDescription: While working on a vehicle repair...",
      "output": "Reasoning: Contorting the body... - InjurySource: Bodily motion..."
  }
  ```

**Prompt Function:**
- Constructs the input using the given `incident_description`.
- Example Prompt:
  ```plaintext
  [INST] <<SYS>>
  Medical coders manages claims by reviewing...
  <</SYS>>

  IncidentDescription: {incident_description}
  [/INST]
  ```

### Validation Steps:

1. **Check Data Type Compatibility:**
   Ensure the dataset format matches the configuration expectations. The "custom" type indicates a tailored structure for specific fields.
   ```yaml
   data_type: "custom"
   ```

### Format validator

In [ ]:
import json

try:
    with open('synthec_data.json') as f:
        data = json.load(f)
    
    for i, entry in enumerate(data):
        assert 'instruction' in entry, f"Missing 'instruction' in entry {i}"
        assert 'input' in entry, f"Missing 'input' in entry {i}"
        assert 'output' in entry, f"Missing 'output' in entry {i}"
    
    print("All entries have 'instruction', 'input', and 'output'.")
except AssertionError as e:
    print(f"AssertionError: {e}")
except Exception as e:
    print(f"An error occurred: {e}")


### Prompt Validator

Verifying that the constructed prompt matches the expected input during fine-tuning.

In [ ]:
import torch
import re

# Test tokenization consistency
tokens1 = tokenizer(prompt(incident_description), return_tensors="pt").input_ids
tokens2 = tokenizer(prompt(incident_description), return_tensors="pt").input_ids
assert torch.equal(tokens1, tokens2), "Tokenization is inconsistent"

# Test inference consistency
output1 = prompt_tok(incident_description)
output2 = prompt_tok(incident_description)
assert output1 == output2, "Inference is inconsistent"

# Test format validation
def validate_output_format(output):
    reasoning_pattern = r"Reasoning: .+ - InjurySource: .+"
    match = re.search(reasoning_pattern, output)
    assert match, "Output format is incorrect"

output = prompt_tok(incident_description)
validate_output_format(output)

# Test token length
input_ids = tokenizer(prompt(incident_description), return_tensors="pt").input_ids
assert input_ids.size(1) <= model.config.max_position_embeddings, "Input token length exceeds the limit"

# Performance test with sample data
test_cases = [
    "While working on a vehicle repair, I had to contort my body...",
    "I was struck by a low-hanging branch from a tree..."
]

for case in test_cases:
    output = prompt_tok(case)
    validate_output_format(output)
    print(output)

# Stress test with edge cases
edge_cases = [
    "",  # Empty input
    "a" * 1000,  # Very long input
    "The quick brown fox jumps over the lazy dog",  # General non-related text
]

for case in edge_cases:
    output = prompt_tok(case)
    validate_output_format(output)
    print(output)
